In [17]:
from decode_time_greedy import extract_routes, find_solution, repair_capacity, repair_distance

In [18]:
import random
def random_permutation(n):
    numbers = list(range(1, n+1))
    random.shuffle(numbers)
    return numbers

def random_binary_vector(n, p):
    vec = []
    for _ in range(n):
        if random.random() < p:   # random() ~ U[0,1)
            vec.append(0)
        else:
            vec.append(1)
    return vec

In [19]:
number_customer = 100
number_truck = 10
number_drone = 10
from data import load_data
problem = load_data(r"data\100customers\r110.txt", 100, 10, 15)
for customer in problem.customer_list:
    print(customer)

Customer(x=35, y=35, demand=0, time=0.0, service=0.0)
Customer(x=41, y=49, demand=10, time=1.625, service=0.008333333333333333)
Customer(x=35, y=17, demand=7, time=0.25, service=0.008333333333333333)
Customer(x=55, y=45, demand=13, time=1.325, service=0.008333333333333333)
Customer(x=55, y=20, demand=19, time=0.8875, service=0.008333333333333333)
Customer(x=15, y=30, demand=26, time=0.25, service=0.008333333333333333)
Customer(x=25, y=30, demand=3, time=0.675, service=0.008333333333333333)
Customer(x=20, y=50, demand=5, time=0.825, service=0.008333333333333333)
Customer(x=10, y=43, demand=9, time=0.7625, service=0.008333333333333333)
Customer(x=55, y=60, demand=16, time=0.6625, service=0.008333333333333333)
Customer(x=30, y=60, demand=16, time=1.2625, service=0.008333333333333333)
Customer(x=20, y=65, demand=12, time=0.4125, service=0.008333333333333333)
Customer(x=50, y=35, demand=19, time=0.475, service=0.008333333333333333)
Customer(x=30, y=25, demand=23, time=0.875, service=0.00833

In [20]:
per_customers = random_permutation(number_customer + number_truck + number_drone -2)
drone_truck_assign = random_binary_vector(number_customer + number_truck + number_drone -2, 0.5)
chromosome = [per_customers, drone_truck_assign]
chromosome = repair_distance(chromosome, problem)
chromosome = repair_capacity(chromosome, problem)
print(chromosome)

[[52, 35, 15, 77, 74, 75, 30, 1, 94, 28, 42, 20, 70, 38, 88, 101, 87, 29, 31, 39, 102, 72, 13, 61, 48, 103, 82, 83, 85, 84, 65, 104, 105, 80, 50, 54, 95, 3, 68, 98, 53, 100, 106, 67, 107, 47, 73, 49, 108, 89, 51, 43, 57, 91, 24, 12, 23, 22, 86, 8, 109, 16, 93, 37, 110, 111, 90, 71, 18, 112, 33, 113, 64, 114, 115, 27, 6, 32, 99, 21, 56, 66, 55, 96, 19, 63, 4, 10, 60, 36, 7, 46, 79, 26, 2, 44, 116, 76, 41, 62, 34, 14, 5, 40, 9, 11, 17, 78, 117, 45, 81, 118, 92, 97, 69, 58, 59, 25], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [21]:
# Chạy thuật toán và in kết quả

assigned_truck_customers, assigned_drone_customers = extract_routes(chromosome, problem)
truck_solutions, drone_solutions = find_solution(assigned_truck_customers, assigned_drone_customers, problem)
while truck_solutions == False:
    idx = chromosome[0].index(drone_solutions)
    chromosome[1][idx] = 0
    chromosome = repair_distance(chromosome, problem)
    chromosome = repair_capacity(chromosome, problem)
    assigned_truck_customers, assigned_drone_customers = extract_routes(chromosome, problem)
    print("Lap lai decode")
    truck_solutions, drone_solutions = find_solution(assigned_truck_customers, assigned_drone_customers, problem)

    
# In kết quả cho các truck
print("Kết quả cho Truck:")
for truck_solution in truck_solutions:
    print(f"  - Khách hàng được giao: {truck_solution.assigned_customers}")
    print(f"  - Thời gian đến: {truck_solution.arrive_time}")
    print(f"  - Thời gian rời đi: {truck_solution.leave_time}")
    print(f"  - Lượng hàng truck nhận: {truck_solution.recived_truck}")
    print(f"  - Lượng hàng drone nhận: {truck_solution.recived_drone}")
    print("-" * 20)

# In kết quả cho các drone
print("\nKết quả cho Drone:")
for i, drone_solution in enumerate(drone_solutions):
    print(f"  - Trip {i+1}:")
    if not drone_solution.trip_list:
        print("    - Không có chuyến bay khả thi.")
        continue

    for j, sortie in enumerate(drone_solution.trip_list):
        print(sortie.assigned_customers)
        print(sortie.arrive_time)
        print(sortie.leave_time)
        print(sortie.recived_drone)
    print("=" * 30)

Lap lai decode
Lap lai decode
Kết quả cho Truck:
  - Khách hàng được giao: [52, 35, 15, 77, 74, 75, 30, 1, 94, 28, 42, 20, 70, 38, 88]
  - Thời gian đến: [ 0.4         1.85833333  4.19166667  5.725       6.65833333  6.79166667
  8.25        8.55833333  9.49166667 10.125      11.18333333 13.04166667
 13.475      15.55833333 17.26666667]
  - Thời gian rời đi: [ 0.40833333  1.86666667  4.2         5.73333333  6.66666667  6.8
  8.25833333  8.56666667  9.5        10.13333333 11.19166667 13.05
 13.48333333 15.56666667 17.275     ]
  - Lượng hàng truck nhận: [ 9.  8.  8. 14.  8. 18. 21. 10. 27. 16.  5.  9.  5. 16.  9.]
  - Lượng hàng drone nhận: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------------------
  - Khách hàng được giao: [87, 29, 31, 39]
  - Thời gian đến: [0.6        2.55833333 3.64166667 5.375     ]
  - Thời gian rời đi: [1.05833333 2.56666667 3.65       5.38333333]
  - Lượng hàng truck nhận: [26.  9. 27. 31.]
  - Lượng hàng drone nhận: [0. 0. 0. 0.]
--------------------
  

In [22]:
for truck_sol in truck_solutions:
    print(problem.check_capacity_truck_contraint(truck_sol))


print("+++++++++++++++++++++++")
for drone_sol in drone_solutions:
    for drone_trip in drone_sol.trip_list:
        print(problem.check_energy_drone(drone_trip))

    print("++++++++++++++++++++++++++++++++++")

True
True
True
True
True
True
True
True
True
True
+++++++++++++++++++++++
True
True
True
++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++
True
True
True
++++++++++++++++++++++++++++++++++
True
++++++++++++++++++++++++++++++++++
True
++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
++++++++++++++++++++++++++++++++++
True
True
True
True
True
True
True
True
True
True
True
++++++++++++++++++++++++++++++++++
True
True
++++++++++++++++++++++++++++++++++
True
True
True
True
True
True
++++++++++++++++++++++++++++++++++


In [23]:
customer = []

for truck_sol in truck_solutions:
    customer = customer + truck_sol.assigned_customers

for drone_sol in drone_solutions:
    for drone_trip in drone_sol.trip_list:
        customer = customer + drone_trip.assigned_customers[1:-1]

print(sum(customer))

5050


In [24]:
print(101*50)

5050
